In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("gapminder.csv")

In [3]:
df.head()

,population,fertility,HIV,CO2,BMI_male,GDP,BMI_female,life,child_mortality,Region
0,34811059.0,2.73,0.1,3.328945,24.59620,12314.0,129.9049,75.3,29.5,Middle East & North Africa
1,19842251.0,6.43,2.0,1.474353,22.25083,7103.0,130.1247,58.3,192.0,Sub-Saharan Africa
2,40381860.0,2.24,0.5,4.785170,27.50170,14646.0,118.8915,75.5,15.4,America
3,2975029.0,1.40,0.1,1.804106,25.35542,7383.0,132.8108,72.5,20.0,Europe & Central Asia
4,21370348.0,1.96,0.1,18.016313,27.56373,41312.0,117.3755,81.5,5.2,East Asia & Pacific


In [4]:
df.isnull().sum() 

population         0
fertility          0
HIV                0
CO2                0
BMI_male           0
GDP                0
BMI_female         0
life               0
child_mortality    0
Region             0
dtype: int64

In [5]:
df.dtypes

population         float64
fertility          float64
HIV                float64
CO2                float64
BMI_male           float64
GDP                float64
BMI_female         float64
life               float64
child_mortality    float64
Region              object
dtype: object

In [19]:
import numpy as np
numerical_df = df.select_dtypes(include=np.number)

In [7]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(16,8))
import seaborn as sns
sns.heatmap(numerical_df.corr(), annot = True)

In [8]:
features_by_filter_method = ["HIV", "GDP", "BMI_female", "child_mortality"]

In [9]:
X = df.drop(["life"],1)
y= df["life"]

In [10]:
X = pd.get_dummies(X)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
col_names = list(X)
X = scaler.fit_transform(X)

C:\Users\padma\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [12]:
X = pd.DataFrame(X)

In [13]:
X.columns = col_names

In [14]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [15]:
from sklearn.model_selection import cross_val_score,GridSearchCV

In [16]:
categorical = list(X)[-6:-1]

In [17]:
categorical

['Region_America',
 'Region_East Asia & Pacific',
 'Region_Europe & Central Asia',
 'Region_Middle East & North Africa',
 'Region_South Asia']

In [18]:
X_cat = X[categorical]

In [132]:
X_num = X[features_by_filter_method]

In [133]:
X = pd.concat([X_cat,X_num],1)

In [134]:
from sklearn.neighbors import KNeighborsRegressor

In [135]:
knn = KNeighborsRegressor(n_neighbors=5)

In [136]:
cross_val_score(knn,X,y,cv = 10, scoring="r2")

array([0.8211049 , 0.74757598, 0.64555004, 0.7049268 , 0.73794806,
       0.80134007, 0.82241066, 0.72112503, 0.90571085, 0.87873789])

In [137]:
params = {"n_neighbors":np.arange(1,16)}

In [138]:
params

{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])}

In [139]:
knn = KNeighborsRegressor()

In [140]:
knn_cv = GridSearchCV(knn,param_grid=params, cv = 10,scoring="r2")

In [141]:
knn_cv.fit(X,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [142]:
best_model = knn_cv.best_estimator_

In [143]:
knn_cv.best_score_

0.7955345662357166

In [55]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X,y, test_size = 0.2, random_state = 42)

In [56]:
knn.fit(X_train,y_train)
y_pred= knn.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))


3.3720792350339215

In [57]:
r2_score(y_test,y_pred)

0.8297554764012687

In [185]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
lin_reg = LinearRegression()
X = df.drop(["life"],1)
y= df["life"]
X = pd.get_dummies(X)
scaler= MinMaxScaler()
col_names = list(X)
X = scaler.fit_transform(X)
X = pd.DataFrame(X)
X.columns = col_names
# X = X[features_by_filter_method]
X_train, X_test, y_train, y_test = tts(X,y, test_size = 0.2, random_state = 42)

/Users/Arunabh/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [186]:
lin_reg.fit(X_train,y_train)
y_pred = lin_reg.predict(X_test)
r2_score(y_test,y_pred)
np.sqrt(mean_squared_error(y_test,y_pred))

2.346972417900389

In [187]:
cross_val_score(lin_reg,X,y,cv = 10)

array([0.7167508 , 0.80157728, 0.81668174, 0.82942625, 0.7418073 ,
       0.93913741, 0.83464389, 0.80949561, 0.95082034, 0.94307859])

In [188]:
ridge = RidgeCV(alphas=np.arange(0.1,2,0.1))

In [189]:
ridge.fit(X,y)

RidgeCV(alphas=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [190]:
ridge.score(X,y)

0.9031834826223613

In [191]:
lin_reg.fit(X,y)
lin_reg.score(X,y)

0.9052398287536055

In [192]:
lasso = LassoCV(alphas=np.arange(0.1,2,0.1))

In [193]:
lasso.fit(X,y)

/Users/Arunabh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LassoCV(alphas=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
    copy_X=True, cv='warn', eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [195]:
lasso.score(X,y)

0.8825658083305495

In [199]:
from sklearn.linear_model import Ridge
ridge = Ridge()

In [200]:
params = {"alpha":np.arange(0.1,2,0.1)}

In [201]:
params

{'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
        1.4, 1.5, 1.6, 1.7, 1.8, 1.9])}

In [202]:
ridge_g_cv =GridSearchCV(ridge, param_grid=params, cv = 10)

In [203]:
ridge_g_cv.fit(X,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [204]:
ridge_g_cv.best_params_

{'alpha': 0.1}